In [ ]:
using Yao
using Plots
using LinearAlgebra
using FFTW
using SymPy

In [ ]:
arg(z) = atan(imag(z),real(z))

# Discrete Fourier transformation
## Fourier transformation
$$
\hat{f}(k) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{\infty} \mathrm{d}x e^{-ixk} f(x)
$$

## Discrete Fourier transformation
$$
\hat{f}(k) = \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1}e^{\frac{2\pi i}{N}j k} f(j)
$$

Let's look at the transformation matrix

In [ ]:
FM(n) = [cis(2π/n*j*k) for j in 0:n-1, k in 0:n-1] ./ sqrt(n) # n is dimention

Bits shifts
```julia
1<<1 == bit"10" == 2^1 == 2
1<<2 == bit"100" == 2^2 == 4
1<<3 == bit"1000" == 2^3 == 8
```

In [ ]:
FM(1<<2)

$$
\mathrm{DFT}(2) = \frac{1}{2}
\begin{pmatrix}
1 &  1 &  1 &  1\\
1 &  e^{i\pi/2} & e^{i\pi} & e^{3i\pi/2}\\
1 & e^{i\pi} &  e^{2i\pi} & e^{6i\pi/2}\\
1 & e^{3i\pi/2} & e^{3i\pi} &  e^{9i\pi/2}
\end{pmatrix} = \frac{1}{2}
\begin{pmatrix}
1 &  1 &  1 &  1\\
1 &  i & -1 & -i\\
1 & -1 &  1 & -1\\
1 & -i & -1 &  i
\end{pmatrix}.
$$

All elements are equal to each other by module. It is actually the matrix of phases

In [ ]:
abs2.(FM(4))

In [ ]:
arg.(FM(4)) .* (180/π)

Check if it is unitary

In [ ]:
round.(FM(4)' * FM(4); digits=2)

__Unitarity__ $\Rightarrow$ it can be a quantum transformation

__Factorization?__ Can the tranformation be decomposed to sequential action to qubits?

(straightforward to prove [see the youtube video](https://www.youtube.com/watch?v=zzUMn3ykey8&ab_channel=AnantVigyan))

## Implementation of the QFT
There are a few, here is one from [_Yao.jl_](https://tutorials.yaoquantum.org/dev/generated/quick-start/2.qft-phase-estimation/)
![figs/qft.png](figs/qft.png)

```julia
A(i, j) = control(i, j=>shift(2π/(1<<(i-j+1))))
B(n, k) = chain(n, j==k ? put(k=>H) : A(j, k) for j in k:n)
qft(n) = chain(B(n, k) for k in 1:n)
```

To understand that we need a few things:
 - simple rotation (shift gate)
 - controlled totation (shift gate applied only if controlled gate in 1)
 - j binary  decomposition
 $$
\begin{align*}
j &= j_{n-1}j_{n}\dots j_{1} j_{0}\\
  &= j_{n-1} 2^{n-1} + \dots + j_{1} 2 + j_{0}
\end{align*}
$$

### Simple rotation
Where $R$ is a $Z$ rotation gate
$$
R_n = \begin{pmatrix}
1 & 0\\
0 & e^{\frac{2\pi i}{2^n}}
\end{pmatrix}:
\qquad
R_2 = \begin{pmatrix}
1 &\\
& e^{\frac{\pi i}{2}}
\end{pmatrix},\quad
R_3 = \begin{pmatrix}
1 &\\
& e^{\frac{\pi i}{4}}
\end{pmatrix}.\quad
\begin{matrix}
0\\
1\\
\end{matrix}
$$
I.e. zero state is not transformed, the state  
$$
\alpha\left| 0 \right\rangle + \beta\left| 1 \right\rangle \xrightarrow[]{R}
\alpha\left| 0 \right\rangle + \beta e^{\frac{2\pi i}{2^n}}\left| 1 \right\rangle
$$

### Controlled rotation
(do rotation only when the controlled gate is 1)
$$
\text{CZ} \Rightarrow
\begin{pmatrix}
1 & & &\\
 & 1& &\\
 & & 1 &\\
& & & e^{\frac{2\pi i}{2^n}}
\end{pmatrix}
\qquad
%\left[
\begin{matrix}
00\\
01\\
10\\
11
\end{matrix}
%\right]
$$

In [ ]:
R, = @vars R
# 
# controlled not
cnot = [1 0 0 0
        0 1 0 0
        0 0 0 1
        0 0 1 0]
# 𝕀 x R
tp1xR = [1 0 0 0
         0 R 0 0
         0 0 1 0
         0 0 0 R];
# R x 𝕀
tpRx1 = [1 0 0 0
         0 1 0 0
         0 0 R 0
         0 0 0 R]
#
# ---Z----x---------x-----
#         |         |
# ---Z----C----Z†---C-----
# 
cnot * inv(tp1xR)* cnot * tp1xR * tpRx1

### Binary representation
$$
\frac{2\pi i}{N} j \rightarrow \frac{2\pi i}{2^n} (j_{n-1}\dots j_{1}j_{0})
=
\frac{2\pi i}{2^n} (2^{n-1} j_{n-1} + \dots 2 j_{1} + j_{0})
$$
we need rotaitons $\frac{2\pi i}{2^{n-k}}j_k$ for every non zero bit $j_k$

## QuFuTr

In [ ]:
# from Yao documentation
A(i, j) = control(i, j=>shift(2π/(1<<(i-j+1)))) # i is controlled gate, j is to be rotated
B(n, k) = chain(n, j==k ? put(k=>H) : A(j, k) for j in k:n) # sequence H R2 R3 ... Rn-1
qft(n) = chain(B(n, k) for k in 1:n) # add the chain to every qubit

In [ ]:
# it gives
arg.(Matrix(mat(qft(2))))

In [ ]:
# it suppose to be 
arg.(FM(4))

In [ ]:
A(j, k) = control(j, k=>shift(2π/(1<<(k-j+1))))
B(n, k) = chain(n, j==k ? put(k=>H) : A(j, k) for j in k:-1:1)
qft(n) = chain(B(n, k) for k in n:-1:1)
nqft(n) = chain(n, qft(n), [swap(n,i,n+1-i) for i in 1:div(n,2)]...)

In [ ]:
nqft(3)

In [ ]:
1 # --------x-------x----H--
  #         |       |
2 # -----x--|----H--R-------
  #      |  |
3 # --H--R--R---------------

In [ ]:
m = mat(nqft(6));

In [ ]:
phis = arg.(Matrix(m))

In [ ]:
heatmap(phis, c=:balance, clim=(-π,π))

In [ ]:
out = product_state(bit"001100") |> nqft(6)

In [ ]:
n_ex = 6
p_ex = 4

In [ ]:
st = sum(product_state(n_ex, p_ex*i) for i in 0:(div(2^n_ex-1, p_ex))) |> normalize!
bar(probs(st))

In [ ]:
out = st |> copy |> nqft(6)
bar(probs(out))

### Measurements

In [ ]:
histogram(Int.(measure(out, nshots=100)),bins=-0.5:1:(2^n_ex))